<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/pf%2Fpre_covid_model/PF_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("StopWords").getOrCreate()
spark = SparkSession.builder.appName("Sentiment").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

# url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/train-data.csv'
url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/sentiment.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment.csv"), sep=",", header=None)

# Show DataFrame
df.show()

+--------+--------------------+
|     _c0|                 _c1|
+--------+--------------------+
|Positive|Blue Shield of Ca...|
|Positive|RSX: Russian Equi...|
|Positive|XLB: Hold Your Po...|
|Positive|Musgrave Minerals...|
|Positive|ViacomCBS to rest...|
|Positive|Oil climbs to one...|
|Negative|Australian regula...|
|Positive|The Wall Street J...|
|Positive|Ora Banda Mining ...|
|Positive|Google Pay Plans ...|
|Negative|Toyota, Honda Ang...|
| Neutral|ZY Investor Alert...|
| Neutral|Calima Energy spu...|
|Positive|These Marijuana S...|
|Positive|Moderna ETFs to R...|
| Neutral|MacroGenics Annou...|
| Neutral|Zymeworks Announc...|
| Neutral|BioXcel Therapeut...|
| Neutral|Merck to Present ...|
| Neutral|BeiGene to Presen...|
+--------+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
| Neutral|BeiGene to Presen...|    88|
+--------+--------------------+------+
only showing top 20 rows



In [ ]:
# Rename columns
df1 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df1.printSchema()
df1.show()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
| Neutral|BeiGene to Pre

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
pos_neg_to_num

StringIndexer_6cde41d13b14

In [ ]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="tokenized")
tokenized = tokenizer.transform(df1)

In [ ]:
# Remove stop words
stop_list = ["Sepp+�l+�", "EUR", ".", "%", "eur", "mn", "m", "?", "$", "#", "@"]
stopremove = StopWordsRemover(inputCol='tokenized',outputCol='stopremove', stopWords=stop_list)
removed_df = stopremove.transform(tokenized)
removed_df.show(truncate=False)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class   |text                                                                                                                                                                         |length|tokenized                                                                                                                                                                                          |stopremove                   

In [ ]:
# Run the hashing term frequency
hashingTF = HashingTF(inputCol="stopremove", outputCol='hash_token')


In [ ]:
# Fit the IDF on the data set
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df1)
cleaned = cleaner.transform(df1)

In [ ]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[43204,50...|
|  1.0|(262145,[31536,58...|
|  1.0|(262145,[619,5765...|
|  1.0|(262145,[57093,58...|
|  1.0|(262145,[27576,38...|
|  1.0|(262145,[15862,27...|
|  2.0|(262145,[6993,206...|
|  1.0|(262145,[38308,63...|
|  1.0|(262145,[22459,27...|
|  1.0|(262145,[66208,84...|
|  2.0|(262145,[3091,942...|
|  0.0|(262145,[102692,1...|
|  0.0|(262145,[9129,233...|
|  1.0|(262145,[16004,22...|
|  1.0|(262145,[27576,49...|
|  0.0|(262145,[18731,22...|
|  0.0|(262145,[31793,45...|
|  0.0|(262145,[1097,216...|
|  0.0|(262145,[2690,275...|
|  0.0|(262145,[26190,27...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|           tokenized|          stopremove|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Negative|2 Super Cheap Hig...|    63|  2.0|[2, super, cheap,...|[2, super, cheap,...|(262144,[12524,19...|(262144,[12524,19...|(262145,[12524,19...|[-652.47336413598...|[0.99996409739599...|       0.0|
|Negative|4 Stocks Insiders...|    29|  2.0|[4, stocks, insid...|[4, stocks, insid...|(262144,[22115,58...|(262144,[22115,58...|(262145,[22115,58...|[-300.30268399599...|[2.75778562254927.

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.672456
